In [1]:
import sys
import timeit
start_all = timeit.default_timer()

!{sys.executable} -m pip install PyAthena

from pyathena import connect

conn = connect(s3_staging_dir='s3://[REDACTED]/sagemakerquery',
               region_name='ap-southeast-1')

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.options.display.max_colwidth = 1000
pd.options.mode.chained_assignment = None
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [3]:
df_sun = pd.read_csv('s3://da-prd-modeling-sagemaker/preprocess/prodrec-item-sun.csv')
df_sun.set_index('item_id', inplace=True)
df_sun_backup = df_sun.copy()

In [4]:
# replace the brand value

df_sun.replace({'chmhierdesc': {
    'Murah': 0,
    'Low Brand': 1,
    'Medium Low Brand': 2,
    'Medium Brand': 3,
    'Medium High Brand': 4,
    'High Brand': 5
}}, inplace=True)

In [5]:
# remove the non-included columns

#not_included = ["deprice", "chmhierdesc", "ZFRAME_BRAND_DISPLAY", "CREATION_TIMESTAMP", "brand"]
#not_included = ["deprice", "ZFRAME_BRAND_DISPLAY", "CREATION_TIMESTAMP", "brand", 'brand_category']
not_included = ["deprice", "zframe_brand_display", "creation_timestamp", "brand", 'brand_category', 'xfactor']
df_sun = df_sun[[a for a in list(df_sun) if a not in not_included]]

In [6]:
# value correction, resulting from wrong scale

df_sun.loc[df_sun['zframe_base_curve'] > 100, 'zframe_base_curve'] = df_sun['zframe_base_curve']/100
df_sun.loc[df_sun['zframe_base_curve'] > 10, 'zframe_base_curve'] = df_sun['zframe_base_curve']/10
df_sun.loc[(df_sun['zframe_temple_length'] > 0) & (df_sun['zframe_temple_length'] < 20), 'zframe_temple_length'] = df_sun['zframe_temple_length']*10
df_sun.loc[(df_sun['zframe_vertical'] > 100), 'zframe_vertical'] = df_sun['zframe_vertical']/10

In [7]:
# replacing 0 with mean for those dimensions that 0 will be impossible

#df_sun.loc[df_sun['ZFRAME_BRIDGE'] == 0, 'ZFRAME_BRIDGE'] =\
#    df_sun[df_sun['ZFRAME_BRIDGE'] != 0]['ZFRAME_BRIDGE'].mean()
df_sun.loc[df_sun['zframe_bridge'] == 0, 'zframe_bridge'] =\
    df_sun[df_sun['zframe_bridge'] != 0]['zframe_bridge'].mean()
#df_sun.loc[df_sun['ZFRAME_DIAGONAL'] == 0, 'ZFRAME_DIAGONAL'] =\
    #df_sun[df_sun['ZFRAME_DIAGONAL'] != 0]['ZFRAME_DIAGONAL'].mean()
df_sun.loc[df_sun['zframe_diagonal'] == 0, 'zframe_diagonal'] =\
    df_sun[df_sun['zframe_diagonal'] != 0]['zframe_diagonal'].mean()
#df_sun.loc[df_sun['ZFRAME_SIZE'] == 0, 'ZFRAME_SIZE'] =\
    #df_sun[df_sun['ZFRAME_SIZE'] != 0]['ZFRAME_SIZE'].mean()
df_sun.loc[df_sun['zframe_size'] == 0, 'zframe_size'] =\
    df_sun[df_sun['zframe_size'] != 0]['zframe_size'].mean()
#df_sun.loc[df_sun['ZFRAME_TEMPLE_LENGTH'] == 0, 'ZFRAME_TEMPLE_LENGTH'] =\
    #df_sun[df_sun['ZFRAME_TEMPLE_LENGTH'] != 0]['ZFRAME_TEMPLE_LENGTH'].mean()
df_sun.loc[df_sun['zframe_temple_length'] == 0, 'zframe_temple_length'] =\
    df_sun[df_sun['zframe_temple_length'] != 0]['zframe_temple_length'].mean()
#df_sun.loc[df_sun['ZFRAME_VERTICAL'] == 0, 'ZFRAME_VERTICAL'] =\
    #df_sun[df_sun['ZFRAME_VERTICAL'] != 0]['ZFRAME_VERTICAL'].mean()
df_sun.loc[df_sun['zframe_vertical'] == 0, 'zframe_vertical'] =\
    df_sun[df_sun['zframe_vertical'] != 0]['zframe_vertical'].mean()

In [8]:
# imputation with mean

#columns = ['ZFRAME_BASE_CURVE', 'ZFRAME_BRIDGE', 'ZFRAME_DIAGONAL', 'ZFRAME_SIZE', 'ZFRAME_TEMPLE_LENGTH', 'ZFRAME_VERTICAL', 'r', 'g', 'b']
columns = ['zframe_base_curve', 'zframe_bridge', 'zframe_diagonal', 'zframe_size', 'zframe_temple_length', 'zframe_vertical', 'r', 'g', 'b']

for c in columns:
    df_sun[c] = df_sun[c].fillna(df_sun[c].mean())

In [9]:
# ordinal encoding + mean imputation
df_sun.replace({'zframe_face_fit': {
        'NR': 0,
        'MD': 1,
        'WD': 2
}}, inplace=True)
df_sun['zframe_face_fit'].fillna(df_sun['zframe_face_fit'].mean(), inplace=True)

In [10]:
# map the nylor metal and nylor into one category
df_sun.replace({'zframe_style': {
        'NM': 'NY'
}}, inplace=True)

In [11]:
# map the ZFRAME_MADE_OF into more generic category
df_sun.replace({'zframe_made_of': {
    'ACE': 'PLA',
    'CAR': 'PLA',
    'CGL': 'PLA',
    'POP': 'PLA',
    'PCI': 'PLA',
    'PUL': 'PLA',
    'TR9': 'PLA',
    'ALU': 'MTL',
    'FLO': 'MTL',
    'STS': 'MTL',
    'TIT': 'MTL',
}}, inplace=True)

In [12]:
# one hot encoding

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
#X = df_sun[['ZFRAME_PRODUCT_SILHOUETTE', 'ZFRAME_MADE_OF', 'ZFRAME_STYLE', 'category', 'chmhierdesc']]
X = df_sun[['zframe_product_silhouette', 'zframe_made_of', 'zframe_style', 'category', 'chmhierdesc']]
X.fillna('na', inplace=True)
enc.fit(X)
df_onehot = pd.DataFrame(enc.transform(X).toarray(), index=df_sun.index,
                         #columns=enc.get_feature_names(['ZFRAME_PRODUCT_SILHOUETTE', 'ZFRAME_MADE_OF', 'ZFRAME_STYLE', 'category', 'chmhierdesc']))
                         columns=enc.get_feature_names(['zframe_product_silhouette', 'zframe_made_of', 'zframe_style', 'category', 'chmhierdesc']))

df_sun = df_sun.merge(df_onehot, left_index=True, right_index=True)
#df_sun.drop(['ZFRAME_FACE_FIT', 'ZFRAME_PRODUCT_SILHOUETTE', 'ZFRAME_MADE_OF', 'ZFRAME_STYLE', 'category', 'chmhierdesc',
#              'ZFRAME_PRODUCT_SILHOUETTE_NA', 'ZFRAME_MADE_OF_NA', 'ZFRAME_STYLE_NA', 'category_NA'], axis=1, inplace=True)
df_sun.drop(['zframe_face_fit', 'zframe_product_silhouette', 'zframe_made_of', 'zframe_style', 'category', 'chmhierdesc',
              'zframe_product_silhouette_na', 'zframe_made_of_na', 'zframe_style_na', 'category_na'], axis=1, inplace=True)

In [13]:
X.fillna('na', inplace=True)
X.isnull().sum()

zframe_product_silhouette    0
zframe_made_of               0
zframe_style                 0
category                     0
chmhierdesc                  0
dtype: int64

In [14]:
# normalize using minmax scaler (min 0, max 1)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0, 1))
df_scaled = pd.DataFrame(scaler.fit_transform(df_sun), columns=df_sun.columns)

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

# TODO: The precision of the result needs to be increased
# there are too many features

cos_sim = cosine_similarity(df_scaled)
indices = pd.Series(df_sun.index)

In [16]:
#set number of recommendations to make
n = 25
def recommendations_with_price(name, cos_sim = cos_sim):
    m = 65 #number of data that is included by the first filter (to minimize run time)
        
    top_n_recommended_sun_inclusive = []
    top_n_recommended_sun = []
    top_n_score_series = []

    idx = indices[indices == name].index[0]
    #cos_sim_idx = cosine_similarity(df_scaled, df_scaled.iloc[idx:(idx+1),:])

    # Create series based on the similarity score
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)
    #score_series = pd.Series(cos_sim_idx.reshape(-1)).sort_values(ascending = False)
    top_n_score_series_inclusive = score_series.iloc[0:(m+1)]
    
    # Get index and create n most similar recommendation
    top_n_indexes_inclusive = list(score_series.iloc[0:(m+1)].index)

    # Create the thresholds for filtering
    recom_index = list(df_sun.index)[idx]
    recom_price = df_sun_backup.loc[recom_index, 'deprice']
    allowed_max = recom_price + 1000000
    min_price = recom_price*0.8
    max_price = recom_price*1.5
    if max_price > allowed_max:
        max_price = allowed_max
       
    # Filter price based on chosen product, put it in list
    for j in top_n_indexes_inclusive:
        #print(j)
        if pd.isnull(recom_price):
            top_n_recommended_sun.append(list(df_sun.index)[j])
            top_n_score_series.append(score_series.loc[j])
        else:
            prod_idx = list(df_sun.index)[j]
            prod_price = df_sun_backup.loc[prod_idx, 'deprice']
            if prod_price > min_price and prod_price < max_price:
                top_n_recommended_sun.append(list(df_sun.index)[j])
                top_n_score_series.append(score_series.loc[j])

    df_recommend = df_sun_backup.loc[top_n_recommended_sun, :]
    df_recommend['score'] = top_n_score_series
    df_recommend = df_recommend.head(n+1)
    df_recommend = df_recommend.index.tolist()
    
    if name in df_recommend:
        df_recommend.insert(0, df_recommend.pop(df_recommend.index(name)))
    else:
        df_recommend.append(name)
        df_recommend.insert(0, df_recommend.pop(df_recommend.index(name)))
    
    return df_recommend

In [17]:
#determine the number of items to put into list
index = df_sun_backup.index
index = index.to_series()
#index = index.iloc[0:1000]

In [18]:
#running the main function for each dataset
import multiprocessing as mp
import timeit

start = timeit.default_timer()

#with mp.Pool(mp.cpu_count()) as pool:
with mp.Pool(16) as pool:
    sun_lists = pool.map(recommendations_with_price, index)

sun_final = pd.DataFrame(sun_lists)

stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  827.2541583340003


In [19]:
# create a backup and new variables for checking
sun_final_backup = sun_final.copy()
sun_final = sun_final.set_index(0)
final_list = sun_final.index.to_list()
index_list = index.to_list()
final_list_dropped = sun_final.index.to_list()

# validate whether there are any duplicates or not
def find_duplicates(L):
    seen = set()
    seen2 = set()
    seen_add = seen.add
    seen2_add = seen2.add
    for item in L:
        if item in seen:
            seen2_add(item)
        else:
            seen_add(item)
    return list(seen2)

find_duplicates(index_list)

[]

In [20]:
sun_final.reset_index(inplace=True)
sun_final.rename(columns={0: "item_id"}, inplace=True)
sun_final.drop(26, axis=1, inplace=True)
sun_final = sun_final.astype('Int64')

In [21]:
sun_final

,item_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,1200000000167,1200000000164,1200000000644,1200000000731,1200000000137,1200000000791,1200000000414,1200000015869,1200000020620,1200000000535,1200000000395,1200000000408,1200000000276,1200000000116,1200000000614,1200000000151,1200000000379,1200000006501,1200000000658,1200000000378,1200000000138,1200000025380,1200000000655,1200000000597,1200000000694,1200000021213
1,1200000000293,1200000010938,1200000015858,1200000016184,1200000023567,1200000022257,1200000025918,1200000015856,1200000025920,1200000023565,1200000021230,1200000020577,1200000018922,1200000010939,1200000016188,1200000006403,1200000011003,1200000025919,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,1200000000448,1200000021274,1200000020066,1200000021261,1200000011208,1200000000768,1200000021264,1200000020548,1200000020033,1200000025207,1200000020029,1200000018676,1200000000758,1200000017038,1200000017039,1200000020023,1200000017088,1200000018771,1200000000554,1200000016289,1200000020025,1200000026461,1200000020031,1200000026440,1200000013925,1200000021238
3,1200000000736,1200000000155,1200000000146,1200000000561,1200000018633,1200000018632,1200000013940,1200000018671,1200000000268,1200000005552,1200000000458,1200000006372,1200000000265,1200000006366,1200000005572,1200000011190,1200000024569,1200000006367,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,1200000000791,1200000000137,1200000015864,1200000000731,1200000000644,1200000000480,1200000000632,1200000000623,1200000000563,1200000000781,1200000000138,1200000000403,1200000000694,1200000025380,1200000000586,1200000000588,1200000000655,1200000000552,1200000000395,1200000000276,1200000000408,1200000010949,1200000000453,1200000026383,1200000021213,1200000000658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30743,1200000025581,1200000004302,1200000025580,1200000025566,1200000025499,1200000003630,1200000003631,1200000003677,1200000003641,1200000003640,1200000003696,1200000003698,1200000003633,1200000003715,1200000003714,1200000003717,1200000004298,1200000004299,1200000003675,1200000003766,1200000003765,1200000004303,1200000025500,1200000025498,1200000025565,1200000003632
30744,1200000025557,1200000025568,1200000025569,1200000003690,1200000025497,1200000025560,1200000004109,1200000025524,1200000025523,1200000025559,1200000025556,1200000025567,1200000025558,1200000025495,1200000025555,1200000025522,1200000025496,1200000003944,1200000003946,1200000004023,1200000003806,1200000024014,1200000003649,1200000007671,1200000007908,<NA>
30745,1200000025501,1200000025502,1200000003965,1200000022750,1200000003998,1200000024797,1200000007970,1200000024799,1200000025503,1200000024862,1200000008985,1200000004023,1200000003946,1200000003944,1200000024014,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
30746,1200000025975,1200000026775,1200000000873,1200000024239,1200000026811,1200000025989,1200000001135,1200000025355,1200000024231,1200000014775,1200000000831,1200000014809,1200000026004,1200000001288,1200000001860,1200000016502,1200000026779,1200000026776,1200000001650,1200000016670,1200000016569,1200000001800,1200000001498,1200000000817,1200000024244,1200000024235


In [22]:
# Save the file

sun_final.to_csv('s3://da-prd-modeling-sagemaker/output/prodrecsun_itemsim.csv', index=False)

stop_all = timeit.default_timer()
print('Time: ', stop_all - start_all)

# Uncomment below to read the file

#sun_final = pd.read_csv('s3://da-prd-modeling-sagemaker/output/prodrecsun-itemsim/prodrecsun-itemsim-final.csv')

Time:  840.7379838639995
